In [1]:
!yc iam create-token

/bin/bash: yc: command not found


In [ ]:
# https://console.cloud.yandex.ru/folders/<идентификатор_каталога>
# https://console.cloud.yandex.ru/cloud/b1g3tb9i9jho3cgthhfj
# https://console.cloud.yandex.ru/folders/b1g9cq6o9a8v973fhg46

In [4]:
import requests
import pprint
import json

token = open("./yandex_gpt_token").read().strip()
headers = {
    "Authorization": f"Bearer {token}",
    "x-folder-id": "b1g9cq6o9a8v973fhg46",
}

instruction_1 = "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить."
instruction_2 = (
    "Перепишите свой предыдущий ответ. Начните каждое предложение с буквы А."
)

data = {
    "model": "general",
    "generationOptions": {
        "partialResults": False,
        "temperature": 0.6,
        "maxTokens": 2000,
    },
    "instructionText": instruction_1,
}

result = requests.post(
    "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
    headers=headers,
    data=json.dumps(data),
)
# "requestText": "string",

# pprint.pprint(result.content)
text = result.json()
text = text["result"]["alternatives"][0]["text"]
text

'Привет, друзья! Недавно я побывал на Гавайях и хочу поделиться с вами своими впечатлениями о путешествии.\n\nГавайи - это настоящий рай на земле, где можно насладиться потрясающими природными пейзажами, прекрасными пляжами и удивительными культурными событиями. Я посетил несколько основных достопримечательностей, которые обязательно стоит посетить, и нашел множество интересных культурных мероприятий, которые стоит попробовать.\n\nПервым делом я посетил знаменитую вулканическую гору Мауна-Кеа, которая является самой высокой точкой на Гавайских островах. С вершины горы открывается потрясающий вид на океан и окружающие острова. Также я посетил национальный парк Вайкики, где можно увидеть множество видов птиц и животных, а также насладиться красивыми пляжами.\n\nОдним из самых интересных культурных событий, которое я посетил, был'

In [23]:
print(text)

Путешествие на Гавайские острова - это настоящее приключение, которое оставит незабываемые впечатления на всю жизнь. Здесь можно насладиться красотой природы, попробовать местную кухню и познакомиться с культурой острова.

Одной из главных достопримечательностей Гавайев является гора Мауна-Кеа. Это самая высокая гора на планете, которая возвышается на 4205 метров над уровнем моря. На вершине горы находится кратер, который был образован в результате извержения вулкана. С вершины открывается потрясающий вид на океан и остров Оаху.

Еще одной интересной достопримечательностью является национальный парк Халеакала. Это вулкан, который до сих пор извергается и считается одним из самых активных вулканов в мире. На территории парка можно увидеть множество гейзеров и горячих источников, которые привлекают туристов со всего мира.

На острове Оаху можно посетить музей искусств Гавайев, который является одним из крупнейших музеев в стране. В музее представлены работы местных художников, а также пр

In [2]:
class YandexGPTAPI:
    def __init__(
        self,
        creds_path="./yandexGPT.json",
        generation_config_path="./yandexGPT_generation_params.json",
    ) -> None:
        self.creds_path = creds_path
        self.generation_config_path = generation_config_path

        self.headers = json.loads(open(self.creds_path).read())
        self.generation_config = json.loads(open(self.generation_config_path).read())

        self.chat_history = []

    def send_instruct(
        self,
        message: str,
    ):
        data = {
            "model": "general",
            "generationOptions": self.generation_config,
            "instructionText": message,
        }

        result = requests.post(
            "https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
            headers=self.headers,
            data=json.dumps(data),
        )
        text = result.json()
        text = text["result"]["alternatives"][0]["text"]
        return text

    def send_chat(self, message: str):
        new_message = {
            "role": "user",
            "text": message,
        }
        self.chat_history.append(new_message)

        data = {
            "model": "general",
            "generationOptions": self.generation_config,
            "messages": self.chat_history,
        }

        result = requests.post(
            "https://llm.api.cloud.yandex.net/llm/v1alpha/chat",
            headers=self.headers,
            data=json.dumps(data),
        )
        text = result.json()
        new_message = text["result"]["message"]
        text = text["result"]["message"]["text"]
        self.chat_history.append(new_message)
        return text

    def clear_chat(self):
        self.chat_history = []

In [ ]:
instruct_model = YandexGPTAPI()
text = instruct_model.send_instruct(message=instruction_1)

print(text)

### chat


In [5]:
instruct_model = YandexGPTAPI()
instructions = [
    # "Напишите настойчивое электронное письмо, чтобы убедить вашего друга интроверта, который не любит публичные выступления, добровольно выступить в качестве приглашенного докладчика на местном мероприятии. Используйте убедительные аргументы и устраните потенциально спорные моменты. Пожалуйста, будьте кратки.",
    # "Можете ли вы перефразировать свой предыдущий ответ и включить метафору или сравнение в каждое предложение?"
    # "Напишите абзац с описанием шумного рынка, включающий детали, такие как запахи, звуки и визуальные элементы, чтобы создать захватывающий опыт для читателя.",
    # "Переработайте свой предыдущий ответ. Начните каждое предложение с последующей буквы алфавита, начиная с B.",
    "Предложите пять отмеченных наградами документальных фильмов с краткими справочными описаниями для начинающих кинематографистов для изучения."
]

for item in instructions:
    text = instruct_model.send_chat(message=item)
    # print(instruct_model.chat_history)
    print(text)
    print("---")

Конечно, вот пять отмеченных наград документальных фильмов, которые могут быть полезны начинающим кинематографистам:

1. "Жизнь в розовом цвете" (2007) - документальный фильм о жизни и карьере певицы Эдит Пиаф. В нем рассказывается о ее детстве, юности, карьере и личной жизни. Фильм получил множество наград, включая "Оскар" за лучший документальный фильм.

2. "Секс, ложь и видео" (2011) - документальный фильм об истории и эволюции порнографии. Он рассказывает о том, как порнография стала популярной в США и как она изменила общество. Фильм получил награду "Эмми" за лучшую документальную программу.

3. "Пешком по Луне" (
---
